In [ ]:
import pandas as pd
from S00_utils import loadPickle, aTime

In [2]:
filepath = "output/S11__2025-04-21-052441.pickle"
dfData = loadPickle(filepath)

In [3]:
def genTsfreshData(row):
    ts = row["ts"]
    t = ts[0]
    s = ts[1]
    data = {"time": t, "s1": s}
    df = pd.DataFrame(data)
    df["id"] = row.name
    return df


temp = dfData.apply(genTsfreshData, axis=1)
dfTsf = pd.concat(temp.values, axis=0)
dfTsf

,time,s1,id
0,0.0,7.049778,E001
1,1.0,7.047202,E001
2,2.0,7.047445,E001
3,3.0,7.050493,E001
4,4.0,7.056179,E001
...,...,...,...
233,233.0,8.185800,E100
234,234.0,8.685793,E100
235,235.0,9.115347,E100
236,236.0,9.457769,E100


In [4]:
from tsfresh import extract_features

dfEx = extract_features(dfTsf, column_id="id", column_sort="time")

Feature Extraction: 100%|██████████| 34/34 [00:03<00:00,  8.74it/s]


In [5]:
dfEx

,s1__variance_larger_than_standard_deviation,s1__has_duplicate_max,s1__has_duplicate_min,s1__has_duplicate,s1__sum_values,s1__abs_energy,s1__mean_abs_change,s1__mean_change,s1__mean_second_derivative_central,s1__median,...,s1__fourier_entropy__bins_5,s1__fourier_entropy__bins_10,s1__fourier_entropy__bins_100,s1__permutation_entropy__dimension_3__tau_1,s1__permutation_entropy__dimension_4__tau_1,s1__permutation_entropy__dimension_5__tau_1,s1__permutation_entropy__dimension_6__tau_1,s1__permutation_entropy__dimension_7__tau_1,s1__query_similarity_count__query_None__threshold_0.0,s1__mean_n_absolute_max__number_of_maxima_7
E001,0.0,0.0,0.0,0.0,2051.527009,14563.576534,0.007642,0.000180,-1.617653e-05,7.099820,...,0.090729,0.090729,0.181214,1.016164,1.323937,1.626878,1.924233,2.215080,NaN,7.149565
E002,0.0,0.0,0.0,0.0,2218.655065,18530.044699,0.061512,0.003188,-6.758523e-05,8.340407,...,0.090729,0.136002,0.136002,0.999357,1.289891,1.576553,1.858735,2.135705,NaN,8.767669
E003,1.0,0.0,0.0,0.0,740.881721,2591.327552,0.253428,0.003639,1.091751e-03,3.005749,...,0.094374,0.094374,0.141461,1.020466,1.343151,1.646825,1.944884,2.236309,NaN,4.750730
E004,1.0,0.0,0.0,0.0,1439.026841,8082.928830,0.307697,-0.007637,6.274485e-04,5.019475,...,0.090729,0.090729,0.181214,0.972656,1.249447,1.522951,1.792686,2.058077,NaN,7.465384
E005,1.0,0.0,0.0,0.0,2401.310325,20714.840537,0.371064,0.005500,-8.575389e-04,8.277214,...,0.079983,0.090729,0.181214,1.014792,1.320328,1.621112,1.916411,2.205328,NaN,10.717452
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
E096,0.0,0.0,0.0,0.0,1109.480551,4909.505081,0.074952,-0.000084,1.642266e-04,4.386030,...,0.126855,0.126855,0.137770,0.959454,1.223566,1.485137,1.743769,1.998992,NaN,4.980681
E097,1.0,0.0,0.0,0.0,1595.645746,11800.677166,0.328932,-0.004092,3.018231e-05,6.857487,...,0.098352,0.147416,0.147416,0.964069,1.232813,1.498993,1.762171,2.021827,NaN,9.562800
E098,0.0,0.0,0.0,0.0,2766.381133,26856.878679,0.184395,-0.001927,9.626387e-04,9.544034,...,0.079983,0.090729,0.181214,0.983164,1.270022,1.553202,1.832161,2.106245,NaN,11.056391
E099,1.0,0.0,0.0,0.0,1172.819578,4981.711521,0.178280,-0.001389,7.950378e-07,3.991749,...,0.090729,0.136002,0.136002,0.966961,1.238130,1.506280,1.770979,2.031718,NaN,5.430081


In [6]:
# Some features have NaN.
from sklearn.impute import SimpleImputer

# Drop columns which contains all NaN.
dfExDrop = dfEx.dropna(axis=1, how="any")

imputer = SimpleImputer(strategy="mean")
exIm = imputer.fit_transform(dfExDrop)
dfExIm = pd.DataFrame(exIm, columns=dfExDrop.columns, index=dfExDrop.index)

In [7]:
from tsfresh.feature_selection.relevance import calculate_relevance_table


def calRelTable(dfEx, target):
    rt = calculate_relevance_table(dfExIm, dfData["y1"])
    rt = rt[rt.relevant].reset_index(drop=True)
    return rt


r1 = calRelTable(dfExIm, dfData["y1"])
r2 = calRelTable(dfExIm, dfData["y2"])
r3 = calRelTable(dfExIm, dfData["y3"])

In [8]:
_dfRt = pd.concat([r1, r2, r3])
dfRt = _dfRt[~_dfRt["feature"].duplicated()]
dfRt

,feature,type,p_value,relevant
0,s1__autocorrelation__lag_8,real,1.898478e-20,True
1,s1__autocorrelation__lag_9,real,2.007450e-20,True
2,s1__autocorrelation__lag_7,real,2.244281e-20,True
3,s1__autocorrelation__lag_6,real,2.372852e-20,True
4,s1__autocorrelation__lag_5,real,3.133353e-20,True
5,s1__autocorrelation__lag_4,real,4.879765e-20,True
6,s1__longest_strike_above_mean,real,5.975334e-20,True
7,s1__autocorrelation__lag_3,real,9.443977e-20,True
8,s1__approximate_entropy__m_2__r_0.7,real,1.053742e-19,True
9,s1__longest_strike_below_mean,real,2.190868e-19,True


In [9]:
dfExRel = dfEx[dfRt["feature"]]
dfExRel

,s1__autocorrelation__lag_8,s1__autocorrelation__lag_9,s1__autocorrelation__lag_7,s1__autocorrelation__lag_6,s1__autocorrelation__lag_5,s1__autocorrelation__lag_4,s1__longest_strike_above_mean,s1__autocorrelation__lag_3,s1__approximate_entropy__m_2__r_0.7,s1__longest_strike_below_mean,...,"s1__agg_autocorrelation__f_agg_""median""__maxlag_40",s1__approximate_entropy__m_2__r_0.5,"s1__agg_autocorrelation__f_agg_""mean""__maxlag_40",s1__ar_coefficient__coeff_3__k_10,s1__approximate_entropy__m_2__r_0.1,s1__lempel_ziv_complexity__bins_3,s1__partial_autocorrelation__lag_4,"s1__fft_coefficient__attr_""abs""__coeff_7","s1__agg_autocorrelation__f_agg_""var""__maxlag_40",s1__spkt_welch_density__coeff_2
E001,-0.305434,-0.519191,-0.074829,0.159896,0.385790,0.590387,14.0,0.762396,0.234735,14.0,...,-0.040299,0.294576,-0.023817,0.183996,0.158567,0.204152,-0.360084,0.293617,0.499488,5.886812e-08
E002,-0.243785,-0.454262,-0.021002,0.202836,0.416423,0.608972,14.0,0.770760,0.222246,14.0,...,0.005711,0.295793,0.017412,0.184370,0.144742,0.203008,-0.344364,6.142373,0.477743,3.643621e-06
E003,-0.329006,-0.543405,-0.095913,0.142612,0.373002,0.582151,14.0,0.758165,0.239496,13.0,...,-0.062521,0.295377,-0.038077,0.184036,0.144268,0.208163,-0.364611,26.783283,0.506435,1.590028e-04
E004,-0.065152,-0.266498,0.138913,0.337187,0.521401,0.683873,16.0,0.817827,0.193755,16.0,...,0.167137,0.283898,0.098749,0.187213,0.137326,0.193662,-0.355441,33.227591,0.460547,7.926165e-04
E005,-0.304881,-0.518177,-0.074836,0.159321,0.384728,0.589003,14.0,0.760924,0.238110,14.0,...,-0.043824,0.288758,-0.022289,0.183978,0.128546,0.192440,-0.357588,11.439470,0.497940,1.462831e-04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
E096,-0.028408,-0.223949,0.168248,0.358374,0.534580,0.690019,16.0,0.818648,0.186632,16.0,...,0.204736,0.291038,0.110039,0.189473,0.123901,0.201581,-0.331990,4.380268,0.469871,2.811382e-06
E097,0.054277,-0.137079,0.243662,0.424124,0.589037,0.732348,17.0,0.848794,0.180211,17.0,...,0.249031,0.277044,0.119486,0.187170,0.132704,0.211207,-0.371926,305.581099,0.486388,1.906617e-04
E098,-0.068519,-0.271997,0.137879,0.338396,0.524483,0.688205,16.0,0.822582,0.196909,15.0,...,0.175901,0.288866,0.095000,0.184988,0.143018,0.190972,-0.369455,29.564227,0.459814,3.510217e-04
E099,0.035297,-0.157917,0.227199,0.410595,0.578612,0.724949,17.0,0.844120,0.184020,17.0,...,0.264798,0.278202,0.118521,0.191178,0.141075,0.190476,-0.368743,6.685749,0.481318,3.133356e-05


In [10]:
cols1 = ["m1", "m2", "m3"]
cols2 = ["y1", "y2", "y3"]
dfTrain = pd.concat([dfData[cols1], dfExRel, dfData[cols2]], axis=1)
dfTrain

,m1,m2,m3,s1__autocorrelation__lag_8,s1__autocorrelation__lag_9,s1__autocorrelation__lag_7,s1__autocorrelation__lag_6,s1__autocorrelation__lag_5,s1__autocorrelation__lag_4,s1__longest_strike_above_mean,...,s1__ar_coefficient__coeff_3__k_10,s1__approximate_entropy__m_2__r_0.1,s1__lempel_ziv_complexity__bins_3,s1__partial_autocorrelation__lag_4,"s1__fft_coefficient__attr_""abs""__coeff_7","s1__agg_autocorrelation__f_agg_""var""__maxlag_40",s1__spkt_welch_density__coeff_2,y1,y2,y3
E001,150.223716,1176.177278,1.142097,-0.305434,-0.519191,-0.074829,0.159896,0.385790,0.590387,14.0,...,0.183996,0.158567,0.204152,-0.360084,0.293617,0.499488,5.886812e-08,55.460434,1.065917,114.578620
E002,102.534268,1483.654982,1.104716,-0.243785,-0.454262,-0.021002,0.202836,0.416423,0.608972,14.0,...,0.184370,0.144742,0.203008,-0.344364,6.142373,0.477743,3.643621e-06,50.640306,1.285666,124.651484
E003,119.890549,1254.897451,2.162773,-0.329006,-0.543405,-0.095913,0.142612,0.373002,0.582151,14.0,...,0.184036,0.144268,0.208163,-0.364611,26.783283,0.506435,1.590028e-04,50.832405,1.154859,57.018054
E004,162.830799,1302.043195,1.308283,-0.065152,-0.266498,0.138913,0.337187,0.521401,0.683873,16.0,...,0.187213,0.137326,0.193662,-0.355441,33.227591,0.460547,7.926165e-04,62.476545,1.025161,132.221218
E005,165.720956,1154.482314,1.566830,-0.304881,-0.518177,-0.074836,0.159321,0.384728,0.589003,14.0,...,0.183978,0.128546,0.192440,-0.357588,11.439470,0.497940,1.462831e-04,57.634438,1.043776,92.160269
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
E096,145.018973,1201.258875,1.944356,-0.028408,-0.223949,0.168248,0.358374,0.534580,0.690019,16.0,...,0.189473,0.123901,0.201581,-0.331990,4.380268,0.469871,2.811382e-06,61.550555,0.912548,105.404790
E097,107.940589,1241.433511,1.827977,0.054277,-0.137079,0.243662,0.424124,0.589037,0.732348,17.0,...,0.187170,0.132704,0.211207,-0.371926,305.581099,0.486388,1.906617e-04,57.735882,0.917145,117.595850
E098,154.310063,1203.547196,1.932303,-0.068519,-0.271997,0.137879,0.338396,0.524483,0.688205,16.0,...,0.184988,0.143018,0.190972,-0.369455,29.564227,0.459814,3.510217e-04,60.867811,0.958479,96.372797
E099,112.805873,1449.500433,1.409830,0.035297,-0.157917,0.227199,0.410595,0.578612,0.724949,17.0,...,0.191178,0.141075,0.190476,-0.368743,6.685749,0.481318,3.133356e-05,57.944120,1.082090,137.838072


In [11]:
filename = f"{aTime('S21')}.xlsx"
filepath = f"output/{filename}"
dfTrain.to_excel(filepath, index=True)

S21__2025-04-21-052727
